In [1]:
import pandas as pd
train = pd.read_csv('atec_anti_fraud_train.csv')
test = pd.read_csv('atec_anti_fraud_test_b.csv')

In [2]:
def data_pre(data): 
    data['day'] = data['date']%100
    return data

temp = train.loc[train.label == -1]
temp.label = 1
train = train.loc[train.label != -1]
train = pd.concat([train,temp])
print train.label.value_counts()

train = data_pre(train)
test = data_pre(test)

/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


0    977884
1     16847
Name: label, dtype: int64


In [3]:
evals_x = train.loc[train.date >= 20171006]
train_x = train.loc[train.date < 20171006]
train_y = train_x.pop('label')
del train_x['id']
del train_x['date']
evals_y = evals_x.pop('label')
del evals_x['id']
del evals_x['date']
res = pd.DataFrame()
del test['date']
res['id'] = test.pop('id')

In [4]:
print evals_y.mean()
print train_y.mean()
print train_x.shape
print evals_x.shape
import lightgbm as lgb
print("LGB test")
params = {
    'boosting':'dart',
    'colsample_bytree':0.8,
    'learning_rate':0.16,
    'metric':'auc',
    'min_child_samples':10,
    'num_leaves':95,
    'object':'binary',
    'reg_alpha':1,
    'reg_lambda':2,
    'seed':2018,
    'silent':1,
    'subsample':1.0,
    'verbose':1
}
train_part = lgb.Dataset(train_x,label=train_y)
evals = lgb.Dataset(evals_x,label=evals_y)
bst = lgb.train(params,train_part, 
              num_boost_round=10000, valid_sets=[train_part,evals], 
              valid_names=['train','evals'], fobj=None,feval=None,feature_name='auto', 
              categorical_feature='auto', early_stopping_rounds=50,
              evals_result=None, verbose_eval=True, learning_rates=None, 
              keep_training_booster=False, callbacks=None)
evals_ypre = bst.predict(evals_x)

0.016881556158616327
0.01699359598909938
(485477, 298)
(509254, 298)
LGB test


/usr/local/lib/python2.7/dist-packages/lightgbm/basic.py:657: UserWarning: silent keyword has been found in `params` and will be ignored. Please use silent argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


[1]	train's auc: 0.938473	evals's auc: 0.943872
Training until validation scores don't improve for 50 rounds.
[2]	train's auc: 0.953044	evals's auc: 0.957313
[3]	train's auc: 0.965887	evals's auc: 0.961573
[4]	train's auc: 0.966828	evals's auc: 0.962298
[5]	train's auc: 0.970524	evals's auc: 0.965258
[6]	train's auc: 0.971899	evals's auc: 0.966061
[7]	train's auc: 0.973409	evals's auc: 0.967138
[8]	train's auc: 0.973565	evals's auc: 0.967449
[9]	train's auc: 0.974043	evals's auc: 0.968787
[10]	train's auc: 0.974923	evals's auc: 0.969203
[11]	train's auc: 0.975136	evals's auc: 0.969012
[12]	train's auc: 0.975475	evals's auc: 0.968749
[13]	train's auc: 0.975727	evals's auc: 0.968729
[14]	train's auc: 0.976397	evals's auc: 0.969056
[15]	train's auc: 0.976844	evals's auc: 0.969016
[16]	train's auc: 0.97771	evals's auc: 0.969243
[17]	train's auc: 0.977927	evals's auc: 0.96914
[18]	train's auc: 0.978263	evals's auc: 0.969046
[19]	train's auc: 0.978952	evals's auc: 0.969356
[20]	train's auc: 

In [5]:
test_ypre = bst.predict(test)
lis = test_ypre
lis_new = []
for li in lis:
    if li<0:
        lis_new.append(0)
    elif li>1:
        lis_new.append(1)
    else:
        lis_new.append(li)
res['score'] = lis_new
print res['score'].describe()
# res.to_csv('result/9_10.csv',index=False)

count    500538.000000
mean          0.018828
std           0.059972
min           0.000000
25%           0.002058
50%           0.003087
75%           0.009402
max           1.000000
Name: score, dtype: float64


In [7]:
# best = pd.read_csv('result/9_10.csv')
import numpy as np
np.corrcoef(best['score'], res['score'])

array([[1.        , 0.99998346],
       [0.99998346, 1.        ]])

In [6]:
from sklearn.metrics import roc_curve
def customedscore(preds, label):
#     label = dtrain.get_label()
    fpr, tpr, thresholds = roc_curve(label,preds,pos_label=1)
    ind = (pd.Series(fpr)-0.001).abs().sort_values().index[0]
    tpr1 = pd.Series(tpr).loc[ind]
    ind = (pd.Series(fpr)-0.005).abs().sort_values().index[0]
    tpr2 = pd.Series(tpr).loc[ind]
    ind = (pd.Series(fpr)-0.01).abs().sort_values().index[0]
    tpr3 = pd.Series(tpr).loc[ind]
    return 'SCORE',tpr1*0.4+tpr2*0.3+tpr3*0.3
                  
print customedscore(bst.predict(evals_x),evals_y)
print bst.predict(evals_x).mean()
print bst.predict(evals_x).min()
print bst.predict(evals_x).max()
evals_ypre = bst.predict(evals_x)
lis = evals_ypre
lis_new = []
for li in lis:
    if li<0:
        lis_new.append(0)
    elif li>1:
        lis_new.append(1)
    else:
        lis_new.append(li)
evals = pd.DataFrame()
evals['lgb1'] = lis_new
print customedscore(evals['lgb1'],evals_y)
print evals['lgb1'].describe()

('SCORE', 0.40264045597301384)
0.01608757184633858
-0.07936958675026909
1.0489580604491922
('SCORE', 0.4027916715133186)
count    509254.000000
mean          0.016105
std           0.068657
min           0.000000
25%           0.001451
50%           0.001720
75%           0.003684
max           1.000000
Name: lgb1, dtype: float64
